Parameters

In [1]:
MODEL_CHECKPOINT = "roberta-base"

Data Pre-Processing and Preparing

Setting up RoBERTa

In [2]:
import torch
from transformers import AutoModel


class RoBERTaClassifier(torch.nn.Module):

    def __init__(self, checkpoint, dropout=0.5):

        super(RoBERTaClassifier, self).__init__()
        
        self.checkpoint = checkpoint
        self.roberta = AutoModel.from_pretrained(checkpoint, num_labels=2)
        self.dropout = torch.nn.Dropout(dropout)
        self.linear2 = torch.nn.Linear(self.roberta.config.hidden_size, 2)


    def forward(self, input_ids, attention_mask, token_type_ids):

        roberta_output = self.roberta(input_ids=input_ids, attention_mask=attention_mask, token_type_ids = token_type_ids)
        dropout_output = self.dropout(roberta_output.pooler_output)
        output = self.linear2(dropout_output)
        
        return output

c:\Users\albbl\Documents\Studium\11_Semester\medtextclassification_repo\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)

config.json: 100%|██████████| 481/481 [00:00<?, ?B/s] 
c:\Users\albbl\Documents\Studium\11_Semester\medtextclassification_repo\.venv\Lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\albbl\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
vocab.json: 100%|██████████| 899k/899k [0

In [5]:
import os
import pandas as pd

from data_preparing import get_dataloader, split_data
from data_preprocessing import xml_to_df
from global_parameters import MAX_LEN, TEST_BATCH_SIZE, TRAIN_BATCH_SIZE, VAL_BATCH_SIZE


xml_files = []
data_path = "../data-querying/results/"
folder_names = ["human_medical_data/", "veterinary_medical_data/"]
for folder in folder_names:
    xml_files.append([f"{data_path}{folder}{xml}" for xml in os.listdir(data_path + folder)])

hum_df, vet_df = xml_to_df(xml_files)

# balance case reports and other text types
vet_case_rep = vet_df[vet_df['text_types'].apply(lambda x: "Case Reports" in x)].sample(frac=1, random_state=42).reset_index(drop=True, inplace=False)
max_num = len(vet_case_rep)
vet_jour_art = vet_df[vet_df['text_types'].apply(lambda x: "Case Reports" not in x)].sample(max_num, random_state=42).reset_index(drop=True, inplace=False)
hum_case_rep = hum_df[hum_df['text_types'].apply(lambda x: "Case Reports" in x)].sample(max_num, random_state=42).reset_index(drop=True, inplace=False)
hum_jour_art = hum_df[hum_df['text_types'].apply(lambda x: "Case Reports" not in x)].sample(max_num, random_state=42).reset_index(drop=True, inplace=False)
hum_df_balanced = pd.concat([hum_case_rep, hum_jour_art])
vet_df_balanced = pd.concat([vet_case_rep, vet_jour_art])

train_set, val_set, test_set = split_data(hum_df_balanced, vet_df_balanced)

train_set["title_abstract"] = train_set[["title", "abstract"]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
val_set["title_abstract"] = val_set[["title", "abstract"]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
test_set["title_abstract"] = test_set[["title", "abstract"]].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

train_dataloader = get_dataloader(train_set.title_abstract, train_set.labels, tokenizer, TRAIN_BATCH_SIZE, MAX_LEN)
val_dataloader = get_dataloader(val_set.title_abstract, val_set.labels, tokenizer, VAL_BATCH_SIZE, MAX_LEN)
test_dataloader = get_dataloader(test_set.title_abstract, test_set.labels, tokenizer, TEST_BATCH_SIZE, MAX_LEN)

  4%|▎         | 3930/105930 [00:00<00:05, 18670.78it/s]

Processing medical field: human_medicine


 33%|███▎      | 34869/105930 [00:01<00:02, 23820.39it/s]

Processing medical field: veterinary_medicine


100%|██████████| 105930/105930 [00:06<00:00, 16319.56it/s]


In [6]:
import os
from transformers import get_linear_schedule_with_warmup

from global_parameters import EPOCHS, LEARNING_RATE, PATH_SAVED_MODELS
from loss_fn import loss_fn
from training import train_model


train = False

model_filename = f"{MODEL_CHECKPOINT}.bin"
filenames = None if not os.path.exists(PATH_SAVED_MODELS) else os.listdir(PATH_SAVED_MODELS)

if filenames and model_filename in filenames and not train:
    model = RoBERTaClassifier(MODEL_CHECKPOINT)
    model.load_state_dict(torch.load(PATH_SAVED_MODELS + model_filename))
    model.to(device)
    print(f"{model_filename} loaded.")
else:
    print("No saved model found or forced to train.")
    
    torch.cuda.empty_cache()
    
    model = RoBERTaClassifier(MODEL_CHECKPOINT)
    model = model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    total_steps = len(train_dataloader) * EPOCHS
    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=0,
      num_training_steps=total_steps
    )
    
    history = train_model(model, train_dataloader, val_dataloader, TRAIN_BATCH_SIZE, loss_fn, optimizer, device, scheduler, EPOCHS)

No saved model found or forced to train.


model.safetensors: 100%|██████████| 499M/499M [00:14<00:00, 34.4MB/s] 
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/28770 [00:00<?, ?it/s]

______________________________
Epoch 0 started.


c:\Users\albbl\Documents\Studium\11_Semester\medtextclassification_repo\.venv\Lib\site-packages\transformers\tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [ ]:
from training import plot


if 'history' in locals():
    history = pd.DataFrame(history)
    data_list = [history['train_acc'], history['val_acc']]
    label_list = ["train accuracy", "validation accuracy"]
    plot(data_list, label_list, "Training history", "Accuracy", "Epoch", [0.95, 1])

In [ ]:
from sklearn.metrics import classification_report

from global_parameters import LABELS_MAP
from predict import predict


test_preds = predict(model, test_set.title_abstract, tokenizer, device)
preds_labels = [torch.argmax(pred).item() for pred in test_preds]
report = classification_report(test_set.labels, preds_labels, target_names=LABELS_MAP.keys())
print(report)

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

from global_parameters import LABELS_MAP


labels = list(LABELS_MAP.keys())
test_classes = [labels[0] if label == 0 else labels[1] for label in test_set.labels]
preds_classes = [labels[0] if label == 0 else labels[1] for label in preds_labels]
disp = ConfusionMatrixDisplay.from_predictions(test_classes, preds_classes, labels=labels, normalize="true", cmap=plt.cm.Blues)
disp.ax_.set_title("Confusion matrix (normalized)")
plt.show()